# Markovitz Portfolio Selection

In [ ]:
using DataFrames
using CSV
using Plots
using Statistics
using Dates

using JuMP
using COSMO

In [ ]:
# Load the data and visualize first rows
data = DataFrames.DataFrame(CSV.File("data/all_stocks_5yr.csv"));
print(first(data, 5))
stocks = Dict()
for (key, subdf) in pairs(groupby(data, :Name))
    stocks[key.Name] = copy(subdf)
end

In [ ]:
# Get a fixed order of the stocks
#stock_names = collect(keys(stocks));
stock_names = unique(keys(stocks));

# randomly select some stocks to reduce the amount of data
N_stocks = 110
stocks_selected = rand(stock_names, N_stocks);

# delete some stocks
for name in stock_names
    if name in stocks_selected
        continue
    end
    delete!(stocks, name)
end;

# adapt stock_names array after stock deletion
stock_names = collect(keys(stocks));
N_stocks = length(stock_names);

In [ ]:
# Visualize evolution of stock changes for some examples
plot(stocks[stock_names[1]].Date, stocks[stock_names[1]].Close / first(stocks[stock_names[1]].Close); label=stock_names[1], legend=:topleft)
plot!(stocks[stock_names[2]].Date, stocks[stock_names[2]].Close / first(stocks[stock_names[2]].Close); label=stock_names[2])
plot!(stocks[stock_names[3]].Date, stocks[stock_names[3]].Close / first(stocks[stock_names[3]].Close); label=stock_names[3])

## Calculate Statistical Quantities

In [ ]:
# Calculate the daily change
for k in keys(stocks)
   select!(stocks[k], :Name, :Date, :Open, :Close, [:Open,:Close] => ByRow((o,c) -> c/o) => :Change)
end

In [ ]:
# plot the daily changes for an example in a histogram
stock_number = 4
histogram(filter(x -> x !== missing, stocks[stock_names[stock_number]].Change), label=stock_names[stock_number])
xlabel!("relative daily change")
ylabel!("counts")

In [ ]:
# ===============================================================================
# Compute the average of the daily changes for each stock
# Hint: As done for plotting the histogram above you should filter out missing values
avg = zeros(N_stocks);
# ===============================================================================

In [ ]:
# define a function to compute the covariance of two distributions s1 and s2
# s1, s2: input arrays containing stock changes
function covariance(s1, s2)

    l = min(length(s1), length(s2))
    # make sure that both stocks have data at the same day!
    gi = filter(i -> s1[i] !== missing && s2[i] !== missing, 1:l) # good indices
    l = length(gi)
    
    covariance = zeros(l,l)
    # ==================================================================================================
    # Compute the covariance
    # ==================================================================================================
    return covariance
end

In [ ]:
# =============================================================================================
# Compute the covariance matrix
covar = zeros(N_stocks ,N_stocks);
# =============================================================================================

In [ ]:
# =============================================================================================
# Compute the Variance for each stock
# Hint: As done for plotting the histogram above you should filter out missing values
variance = zeros(N_stocks);
# =============================================================================================

In [ ]:
# Show annualized standard deviation and average
fig = scatter([], [], xlabel="Annualized Standard Deviation", ylim=(0.75,1.7), xlim=(-0.01,.4), ylabel="Annualized Expected Value", labels=:none, legend=:topleft)
for k=1:N_stocks 
    scatter!([sqrt(variance[k]) * sqrt(252)], [avg[k]^252], labels=:none, markercolor=:blue)
end
scatter!([0.0], [1.02], label="Government Bonds", markercolor=:red)

In [ ]:
# Compute correlation between stocks and visualize in a heatmap
corre = zeros(N_stocks,N_stocks)
for i=1:N_stocks , j=1:N_stocks
    corre[i,j] = covar[i,j] / (sqrt(variance[i])*sqrt(variance[j]))
end

heatmap(corre, yflip=true, aspect_ratio=:equal, size=(300,300))

## Optimize Portfolio

In [ ]:
# =============================================================================================
# Optimize your portfolio by using the COSMO Optimizer (Quadratic solver).
# Choose your favourite form of the optimization problem (lecture 5, slide 21).
# Depending on your choice, try different values for the minimum expected change, the maximum variance 
# or the risk aversion parameter

model = Model(COSMO.Optimizer)
@variable(model, w[1:N_stocks])
#@constraint(model, ...)
#@objective(model, Max, ...)
#optimize!(model)

# =============================================================================================

w_best = value.(w[1:N_stocks]);

In [ ]:
indices = sortperm(w_best, rev=true);
println(w_best[indices[1:5]],"\n",stock_names[indices[1:5]])

In [ ]:
#plot your optimized portfolio and see how your portfolio performs in comparison to single stock investments

fig = scatter([], [], xlabel="Annualized Standard Deviation", ylim=(0.75,1.7), xlim=(-0.01,.4), ylabel="Annualized Expected Value", labels=:none, legend=:topleft)
for k=1:N_stocks
    scatter!([sqrt(variance[k]) * sqrt(252)], [avg[k]^252], labels=:none, markercolor=:blue)
end
scatter!([0.0], [1.02], label="Government Bonds", markercolor=:red)

risk = w_best'*covar*w_best
ret = avg'*w_best
scatter!([sqrt(risk) * sqrt(252)], [ret^252], labels="Selected Portfolio", markercolor=:green, marker=:+, markersize=5,linewidth=20)


# Exercise 12.2: The inverted pendulum

**Remark to generate the animations**
* Install the ffmpeg package, e.g. by conda install ffmpeg!!
* Beware that existing gifs might not be overwritten automatically. Make sure to always give a new name to the `animate_pendulum` function or delete the existing file.


In [ ]:
using Luxor

import LinearAlgebra
LA = LinearAlgebra;

In [ ]:
# define some global variables

# pendulum mass in kg
const m = 2
# cart mass in kg
const c = 8
# pole length in m
const l = 0.5
# earth acceleration in m/s²
const g = 9.81

# number of time steps for prediction horizion in MPC
T = 100
# length of time steps in s
Δt = 0.03
# time discount to weight objective function different depending on the amount of time passed
time_discount = 0.1

# number of variables of interest in optimization problem: 4 state space variables + 1 control
n = 5
# number of constraints in optimization problem
k = 4;

In [ ]:
# Function for plotting pendulum animation
# Needs the ffmpeg package!! and a folder tmp
# Beware that existing gifs might not be overwritten automatically.
# Make sure to always give a new name or delete the existing
# p: array of cart positions for all times
# θ: array of pendulum positions for all times
# name: name of gif-file containing the animation
function animate_pendulum(p,θ,name)
    
    nt = length(p)
    
    # iterate over all times and generate a png per time step
    for i=1:nt
    @png begin
        background("white")
        factor = 400
        p1 = Point(p[i]*factor, 0)
        p2 = Point(sin(θ[i])*factor*l,-cos(θ[i])*factor*l) + p1
        origin()
        setline(10)
        sethue("red")
        line(p1, p2, :stroke)
        sethue("gold")
        line(Point(-800, 0), Point(800, 0), :stroke)
        sethue("black")
        Luxor.box(p1, 80, 40, :fill)
        circle(p2, 20, :fill)
        preview()
            end 2000 600 "tmp/"* name * "_" * lpad(i,2,"0") * ".png"
    end
    
    # combine png images to gif
    loadpath = "tmp/" * name * "_"
    run(`ffmpeg -framerate 15 -i $loadpath"%02d.png" "$name.mp4"`)
    
end

## Exercise 12.2b: Pendulum simulation

\begin{align*}
\vec x_{t+1} = A_t \vec x_t + \vec b_t  u_t + \vec c_t\\
\end{align*}

In [ ]:
# returns the quantities needed to calculate x_t+1 from time x_t and u_t
function At_bt_ct(θ,dθ)
    # acceleration linear in xt
    ddθ_fixed = Δt * ((c+m)*g*sin(θ)-m*l*sin(θ)*cos(θ)*(dθ^2)) / (l*(c+m*sin(θ)^2))
    ddp_fixed = Δt * (-m*g*sin(θ)*cos(θ)+m*l*sin(θ)*dθ^2) / (c+m*sin(θ)^2)
    
    # acceleration linear in u
    ddθ_u = Δt * (-cos(θ)) / (l*(c+m*sin(θ)^2))
    ddp_u = Δt / (c+m*sin(θ)^2)
    
    At = zeros(4,4)
    bt = zeros(4)
    ct = zeros(4)
    
    # ====================================================================================================
    # Construct the matrix At and the vectors bt and ct 
    # ====================================================================================================
    
    return At, bt, ct
end

In [ ]:
# simulate the pendulum movement
# x0: initial state vector
# u: array of control for each time step
# xt: array with state vector for each time step
function simulate_pendulum(x0,u)
    x_new = copy(x0)
    
    # number of time steps extracted from the length of control vector
    nt = length(u)
    xt = Vector(undef, nt)
    
    # iterate over time steps
    for t in 1:nt
        x = x_new
        At, bt, ct = At_bt_ct(x[1],x[3])
        x_new = At*x + bt*u[t] + ct
        xt[t] = x_new
    end
    return xt
end

In [ ]:
# set number of time steps of size Δt for which to simulate pendulum movement
time_sim = 200
x_arr = simulate_pendulum([0.75*pi,0,0,0],zeros(time_sim));

In [ ]:
p_initial = map(x->x[2],x_arr)
θ_initial = map(x->x[1],x_arr)

animate_pendulum(p_initial,θ_initial,"pendulum")

## Exercise 12.2d: Model predictive control

From the lecture we know the problem definition where we want to minimize the kinetic energy of the system (carriage + pendulum) and maximize the potential energy of the pendulum:

\begin{align*}
      \text{Kinetic Energy:} & \quad \begin{aligned}[t]
          e_{kin,kart}(\vec x) &= \frac{1}{2} c \dot p^2\\
          e_{kin,pend}(\vec x) &= \frac{1}{2} m \Big[(\dot p + l \color{red}{\sin(\theta)} \dot\theta)^2 + ( l \color{red}{\cos(\theta)} \dot\theta)^2 \Big]
        \end{aligned}\\
    \text{Potential Energy:} & \quad e_{\mathrm{pot}}(\vec x) = m\, l\, \color{red}{\cos(\theta)}\\
    &f_{\mathrm{state}}(\vec x_t) = \tilde e_{\mathrm{kin,cart}}(\vec x_t) + \tilde e_{\mathrm{kin,pend}}(\vec x_t) - \tilde e_{\mathrm{pot}}(\vec x_t)
  \end{align*}
  
 To ensure that the objective $f_{\mathrm{state}}(\vec x_t)$ is a convex function we have to replace the trigonometric functions $\cos$ and $\sin$ (red terms). We use a simulation run of the current control flow and the pendulum dynamics to generate a series of $\theta$ and $p$. We use those values in order to fix and approximate the $\cos$ and $\sin$ terms before optimizing $x_t$ and $u_t$. We recompute the control flow after each time step to keep the deviations between this approximation and reality as small as possible. 
 
As you might have noticed the potential energy that would lead to an upright pendulum is no longer dependent on $\theta$ or $p$, at least not the $\theta$ and $p$ that we optimize over.
To make the approximation work we have to introduce additional incentives. The original $e_{\mathrm{pot}}(\vec x) = m\, l\, \cos(\theta)$ would be maximized for $\theta = 0$. We therefore introduce $\theta^2$ as part of our objective:

\begin{align*}
    \min_{\substack{\vec u_0,\dots \vec u_{T-1},\\\vec x_1,\dots, \vec x_T}} \quad & \sum_{t=1}^{T} \left[ f_{\mathrm{state}}(\vec x_t) + \vec u_t^\top R \vec u_t + \theta_t^2\right]\\
  \end{align*}
  
 Additionally we would like to keep the pendulum centered to the screen. We therefore also require $p$ to be close to 0. Hence, we add the term $p^2$:
 
 \begin{align*}
    \min_{\substack{\vec u_0,\dots \vec u_{T-1},\\\vec x_1,\dots, \vec x_T}} \quad & \sum_{t=1}^{T} \left[ f_{\mathrm{state}}(\vec x_t) + \vec u_t^\top R \vec u_t + \theta_t^2+p_t^2\right]\\
  \end{align*}
  
Not all conditions are equally important at all points in time. For example: we want to use fast movement that is rich in kinetic energy to swing the pendulum upright in the beginning, but use only small movements to hold it there. We therefore also introduce a time series discount factor $\gamma$:

\begin{align*}
    \min_{\substack{\vec u_0,\dots \vec u_{T-1},\\\vec x_1,\dots, \vec x_T}} \quad & \sum_{t=1}^{T} \left[ (f_{\mathrm{state}}(\vec x_t) + \vec u_t^\top R \vec u_t + p_t^2)\cdot \gamma t +\theta_t^2\right]\\
  \end{align*}
  
This objective will be expressed by $\tilde{x_T^\top} Q \tilde{x_T}$ with the cost matrix $Q$ and
\begin{align*}
\tilde{\vec x}_T = \begin{pmatrix}
\theta_1\\
p_1\\
\dot{\theta_1}\\
\dot{p_1}\\
u_1\\
\theta_2\\
p_2\\
\vdots\\
\dot{\theta_T}\\
\dot{p_T}\\
u_T
\end{pmatrix}
\end{align*}.

This means

\begin{align*}
    \sum_{t=1}^{T} \left[ (f_{\mathrm{state}}(\vec x_t) + \vec u_t^\top R \vec u_t + p_t^2)\cdot \gamma t +\theta_t^2\right] =  \tilde{x_T^\top} Q \tilde{x_T} \\
  \end{align*}


In [ ]:
# input is the state and the previously computed control u
# output is the control in this period
function mpc_pendulum_policy(x0,u_old)
      
    xt = simulate_pendulum(x0,u_old)
   
    # construct Matrix A_tilde and b_tilde for the equality constraints

    # kronecker product of two matrices --> new matrix has size = (T*k,T*n)
    A_tilde = kron(Matrix{Float64}(LA.I,T,T), -Matrix{Float64}(LA.I, k, n))
    b_tilde = zeros(k*T)
    
    b_tilde[1:length(x0)] = -x0 # Set first elements of b_tilde with x0
    
    for t=1:T-1
        # calculate At, bt and ct for each time step
        A_tmp, b_tmp, c_tmp = At_bt_ct(xt[t][1],xt[t][3])
        # construct Matrix A_tilde and b_tilde
        A_tilde[(t*k)+1:(t+1)*k,((t-1)*n)+1:t*n] = hcat(A_tmp,b_tmp)
        b_tilde[(t*k)+1:(t+1)*k] = -c_tmp
    end

    
    #calculate the cost matrix Q 
    Q = zeros(n*T,n*T)
    for t=1:T-1
        θ = xt[t][1]
        c = cos(θ)
        s = sin(θ)
        #Our first objective is to get the pendulum upright and then hold that position in the centre of the line with at best not movement
        #not all objectives are similar important over time. time_discount*t enables us to reweight costs based on the passage of time.
        #make the pendulum upright and centered
        Q[((t-1)*n)+1,((t-1)*n)+1] = 100
        Q[((t-1)*n)+2,((t-1)*n)+2] = time_discount*t
        
        #minimize kinetic energy in the system
        # ============================================================================================
        # fill the matrix Q with the terms of the objective function that are representing the kinetic
        # energy ot the system
        Q[((t-1)*n)+3,((t-1)*n)+3] = 0
        Q[((t-1)*n)+4,((t-1)*n)+4] = 0
        Q[((t-1)*n)+3,((t-1)*n)+4] = 0
        # ============================================================================================
        
        #punish for large control values, if zero we might break the pendulum simulation, but you are encouraged to experiment
        Q[((t-1)*n)+5,((t-1)*n)+5] = 0.001
    end
    
    
    # ============================================================================================
    # setup the optimization problem and solve it using the COSMO solver 
    # define the objective and constraints using the JuMP package
    # ============================================================================================
    
    
    # ============================================================================================
    # extract the sequence of control values from the solution
    u_new = zeros(T)
    
    # ============================================================================================
    return u_new
end

In [ ]:
# Run the model predictive control
# x0: initial pendulum state
# time_sim: time for which the simulation should be run. This NOT the prediction horizon!
function run_mpc(x0,time_sim)
    
    # trace of pendulum state (rows 1-4) and control values (row 5)
    trace = zeros(length(x0)+1, time_sim) # the last entry at each t is the control value u
    
    # vector storing control values for the prediction horizon
    U = zeros(T)

    # copy of inital state
    x = copy(x0)
    
    # iterate over time steps
    for t=1:time_sim
        # compute the control by assuming the last control sequence as the starting point
        U[:] = mpc_pendulum_policy(x, U)

        # store the trace of all pendulum state and control for each time step
        trace[:,t] = vcat(x, U[1])

        # propagate pendulum one time step
        xt = simulate_pendulum(x,U[1]) 
        
        x[:] = xt[1]
    end
    return trace
end

In [ ]:
# initial pendulum state
x0 = [pi, 0, 0, 0]
time_sim = 300
trace = run_mpc(x0, time_sim)

In [ ]:
p_optimized = trace[2,:]
θ_optimized = trace[1,:]

animate_pendulum(p_optimized,θ_optimized,"mpc_pendulum")